In [21]:
PROJ=0
NAME=1
NAME_EN=2
ORG=3
ORG_EN=4
DEPT=5
TITLE=6
EMAIL=7
PHONE=8
MOBILE=9
POSITION=10

lq="{"
rq="}"

import csv
import os

if __name__ == '__main__':
    with open('pairlabs-pi3.csv', newline='') as csvfile:
        lines = csv.reader(csvfile, delimiter=',')
        next(lines, None)  # skip the headers
        projects = set()
        organizations = set()
        professors = dict()
        participation = []
        for line in lines:
            proj = '/'.join(line[PROJ].split('/')[1:])
            projects.add(proj)
            organizations.add(line[ORG]+','+line[ORG_EN])
            professors[line[NAME]]=line[NAME_EN:POSITION+1]
            participation.append((proj,line[NAME],line[POSITION]))
    csvfile.close()
    with open('first_import_cmd', 'w') as arango_cmd:
        arango_cmd.write("db._dropDatabase('test'); \n")
        arango_cmd.write("db._createDatabase('test');\n")
        arango_cmd.write("db._useDatabase('test');\n")
        arango_cmd.write("db._create('person');\n")
        arango_cmd.write("db._create('proj');\n")
        arango_cmd.write("db._create('org');\n")
        arango_cmd.write("db._createEdgeCollection('participation');\n")
        arango_cmd.write("db._createEdgeCollection('work');\n")
        for org in organizations:
            org_ch = org.split(',')[0]
            org_en = org.split(',')[1]
            statement = "db.org.save({2} name : '{0}', name_en : '{1}'{3});\n".format(org_ch,org_en,lq,rq)
            arango_cmd.write(statement)
        for proj in projects:
            statement = "db.proj.save({2} title : '{0}', keyword : '{1}'{3});\n".format(proj,'DL,CNN,RNN',lq,rq)
            arango_cmd.write(statement)
        for key,value in professors.items():
            statement = "var person_id = db.person.save({4} name : '{0}', name_en : '{1}', email : '{2}', expertise : '{3}'{5})._id;\n".format(key,value[0],value[5],'DL,CNN,RNN',lq,rq)
            arango_cmd.write(statement)
            statement =  "var org_id = db._query(`for school in org filter school.name=='{0}' return school._id`).toArray()[0];\n".format(value[1])
            statement += "db.work.save(person_id, org_id,{2} department:'{0}', title:'{1}'{3});\n".format(value[3],value[4],lq,rq)
            arango_cmd.write(statement)
        for t in participation:
            statement = "var person_id = db._query(`for per in person filter per.name=='{0}' return per._id`).toArray()[0];\n".format(t[1])
            statement+= "var proj_id = db._query(`for pro in proj filter pro.title=='{0}' return pro._id`).toArray()[0]; \n".format(t[0])
            statement+= "db.participation.save(person_id, proj_id, {1}position : '{0}' {2}  );\n".format(t[2],lq,rq) 
            arango_cmd.write(statement)
    arango_cmd.close()

    f = os.popen(" arangosh --server.endpoint tcp://192.168.180.140:8529  --server.password 'pair' --javascript.execute first_import_cmd; ")
    print('first import OK')

first import OK 13


In [31]:
PI=0
COUNTRY=1
CITY=2
CO_TYPE=3
ORG=4
CO_NAME=5
DEPARTMENT=6
TITLE=7
EMAIL=8

LQ = "{"
RQ = "}"

import csv
import os

if __name__ == '__main__':
    with open('pairlabs_cooperation.csv', newline='') as csvfile:
        lines = csv.reader(csvfile, delimiter=',')
        next(lines, None)  # skip the headers
        organizations = dict()
        cooperation_object = dict()
        cooperation = []
        for line in lines:
            organizations[line[ORG]]=line[COUNTRY]
            cooperation_object[line[CO_NAME]]=line[PI:EMAIL+1]
            cooperation.append((line[CO_TYPE],line[PI],line[CO_NAME]))
    csvfile.close()
    with open('second_import_cmd', 'w') as arango_cmd:
        arango_cmd.write("db._useDatabase('test');\n")
        arango_cmd.write("db._create('team');\n")
        arango_cmd.write("db._createEdgeCollection('team_in_org');\n")
        arango_cmd.write("db._createEdgeCollection('cooperation_person');\n")
        arango_cmd.write("db._createEdgeCollection('cooperation_team');\n")
        for key,value in organizations.items():
            statement = 'db.org.save({2} name : "{0}" , conuntry : "{1}" {3});\n'.format(key,value,LQ,RQ)
            arango_cmd.write(statement)
        for key,value in cooperation_object.items():
            statement =  'var org_id = db._query(aql`for school in org filter school.name=="{0}" return school._id`).toArray()[0];\n'.format(value[ORG])
            if value[CO_TYPE]=="person":
                statement += 'var person_id = db.person.save({3} name : "{0}", email : "{1}", expertise : "{2}"{4})._id;\n'.format(key,value[EMAIL],'DL,CNN,RNN',LQ,RQ)
                statement += 'db.work.save(person_id, org_id,{2} department:"{0}", title:"{1}"{3});\n'.format(value[DEPARTMENT],value[TITLE],LQ,RQ)
            elif value[CO_TYPE]=="team":
                statement += 'var team_id = db.team.save( {2} name : "{0}" , email : "{1}" {3})._id;\n'.format(key,value[EMAIL],LQ,RQ)
                statement += 'db.team_in_org.save(team_id, org_id,{2} department:"{0}", title:"{1}"{3});\n'.format(value[DEPARTMENT],value[TITLE],LQ,RQ)
            arango_cmd.write(statement)
        for t in cooperation:
            statement = 'var pi_id = db._query(aql`for pi in person filter pi.name=="{0}" return pi._id`).toArray()[0];\n'.format(t[1])
            if t[0]=="person":
                statement+= 'var cooperator_id = db._query(aql`for coo in person filter coo.name=="{0}" return coo._id`).toArray()[0];\n'.format(t[2])
                statement+= 'db.cooperation_person.save(pi_id,cooperator_id, { });\n'
            elif t[0]=="team":
                statement+= 'var team_id = db._query(aql`for te in team filter te.name=="{0}" return te._id`).toArray()[0];\n'.format(t[2])
                statement+= 'db.cooperation_team.save(pi_id,team_id,{ });\n'
            arango_cmd.write(statement)
    arango_cmd.close()
    
    f = os.popen(" arangosh --server.endpoint tcp://192.168.180.140:8529  --server.password 'pair' --javascript.execute second_import_cmd; ")
    print('second import OK 222311!')

second import OK 222311!


In [40]:
from arango import ArangoClient
import http.client, urllib.request, urllib.parse, urllib.error, base64
from argparse import ArgumentParser
import json
import os

import traceback
import time

LQ = "{"
RQ = "}"

#headers = {'Ocp-Apim-Subscription-Key': '0cf2c61493ec498daa84808b6522d1f9'}
headers = {'Ocp-Apim-Subscription-Key': '4a1a9fc9d3b246249c0230f76367bc35'}

conn = http.client.HTTPSConnection('api.labs.cognitive.microsoft.com')


def mag_evaluation(exprs):

    params = urllib.parse.urlencode({
            'expr': exprs,
            'model': 'latest',
            'count': '300',
            'offset': '0',
            'orderby': '',
            'attributes': 'Id,Ti,Y,D,L,CC,AA.AuN,AA.DAuN,AA.AuId,AA.AfN,AA.AfId,AA.AfId,F.FN,F.FId',
        })
    conn.request("GET", "/academic/v1.0/evaluate?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    json_data = json.loads(data.decode('UTF-8'));  #to list type
    #conn.close()
    time.sleep(1)
    return json_data


if __name__ == "__main__":

    print('ini..1')
    f = os.popen(" arangosh --server.endpoint tcp://192.168.180.140:8529  --server.password 'pair' --javascript.execute first_import_cmd; ")
    time.sleep(3)
    print('ini..2')
    f = os.popen(" arangosh --server.endpoint tcp://192.168.180.140:8529  --server.password 'pair' --javascript.execute second_import_cmd; ")
    time.sleep(3)

    #########setup name_list from arangodb###############
    arangodb_client = ArangoClient(protocol='http', host='e1.pairlabs.ai', port=8529)
    db = arangodb_client.db('test', username='root', password='pair')
    #persons = db.aql.execute('for per in person for part in participation filter part._from==per._id return distinct per.name')
    sta = 'for per in person return {0}en : per.name_en, ch : per.name{1}'.format(LQ,RQ)
    #sta = 'for per in person for part in participation filter part._from==per._id return distinct {0}en : per.name_en, ch : per.name{1}'.format(LQ,RQ)
    persons = db.aql.execute(sta)

 
    name_list = []
    for result in persons:  #Cursor to string into list
        name_list.append(result)
    print(str(len(name_list)))

ini..1
ini..2
148


In [42]:
from arango import ArangoClient
import http.client, urllib.request, urllib.parse, urllib.error, base64
from argparse import ArgumentParser
import json
import os

import traceback
import time

LQ = "{"
RQ = "}"

#headers = {'Ocp-Apim-Subscription-Key': '0cf2c61493ec498daa84808b6522d1f9'}
headers = {'Ocp-Apim-Subscription-Key': '4a1a9fc9d3b246249c0230f76367bc35'}

conn = http.client.HTTPSConnection('api.labs.cognitive.microsoft.com')


def mag_evaluation(exprs):

    params = urllib.parse.urlencode({
            'expr': exprs,
            'model': 'latest',
            'count': '300',
            'offset': '0',
            'orderby': '',
            'attributes': 'Id,Ti,Y,D,L,CC,AA.AuN,AA.DAuN,AA.AuId,AA.AfN,AA.AfId,AA.AfId,F.FN,F.FId',
        })
    conn.request("GET", "/academic/v1.0/evaluate?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    json_data = json.loads(data.decode('UTF-8'));  #to list type
    #conn.close()
    time.sleep(1)
    return json_data


if __name__ == "__main__":

    print('import 1..')
    f = os.popen(" arangosh --server.endpoint tcp://192.168.180.140:8529  --server.password 'pair' --javascript.execute first_import_cmd; ")
    time.sleep(3)
    print('import 2..')
    f = os.popen(" arangosh --server.endpoint tcp://192.168.180.140:8529  --server.password 'pair' --javascript.execute second_import_cmd; ")
    time.sleep(3)
    print('OK\n')

    #########setup name_list from arangodb###############
    arangodb_client = ArangoClient(protocol='http', host='e1.pairlabs.ai', port=8529)
    db = arangodb_client.db('test', username='root', password='pair')

    #sta = 'for per in person return {0}en : per.name_en, ch : per.name{1}'.format(LQ,RQ)
    sta = 'for per in person for part in participation filter part._from==per._id return distinct {0}en : per.name_en, ch : per.name{1}'.format(LQ,RQ)
    persons = db.aql.execute(sta)

 
    name_list = []
    for result in persons:  #Cursor to string into list
        name_list.append(result)
    print('PI & CO-PI : ' + str(len(name_list)))

    fake_list = []
    for per in name_list[0:10]:
        fake_list.append(per)
    #####################################################
    print('arango get OK!')
    print('writing .js file ......')

    with open('paper_cmd', 'w') as arango_cmd:
        arango_cmd.write("db._useDatabase('test');\n")
        arango_cmd.write("db._create('paper');\n")
        arango_cmd.write("db._createEdgeCollection('author_of')\n\n")
        arango_cmd.write("var person_id,paper_id,org_id\n\n")

        for name_pair in name_list:
            
            name_en = name_pair['en']
            name_ch = name_pair['ch']

            paper_cnt=0
            friend_num=0
            affiliation_num = 0 
            emp_affiliation = 0


            ORG_EN = db.aql.execute('for per in person filter per.name_en=="{0}" for org in 1..1 outbound per work return org.name_en'.format(name_en))
            org_en = ORG_EN.pop()  #arango.Cursor pop to string
            exprs = "AND(Composite(AND(AA.AuN='{0}',AA.AfN='{1}')),Y>2010)".format(name_en.lower(),org_en.lower())  # Composite(AND(AA.AuN='nenfu huang',AA.AfN='national tsing hua university'))

            json_data = mag_evaluation(exprs)
        
            try:
                #print(name_en + " : ")
                statement = '\n\n//---- {0} ----\n'.format(name_en)
                arango_cmd.write(statement)
                for ENTITY in json_data['entities']:  #save paper
                    paper_cnt +=1
                    MAG_id       = ENTITY['Id']         #save attr
                    title        = ENTITY['Ti']         #save attr
                    try : language = ENTITY['L']          #save attr
                    except: pass
                    year         = ENTITY['Y']          
                    date         = ENTITY['D']          #save attr
                    citation_cnt = ENTITY['CC']         #save attr
                    statement_1 = 'if (!(paper_id=db._query(`for pap in paper filter pap.MAG_id=="{0}" return pap._id`).toArray()[0])) {1}\n'.format(MAG_id,LQ)
                    statement_1 += '    paper_id = db.paper.save( {6} MAG_id:"{0}", title:"{1}", language:"{2}", year:"{3}", date:"{4}", citation_cnt:"{5}", fields:['.format(MAG_id,title,language,year,date,citation_cnt,LQ)
                    try : 
                        for FIELD in ENTITY['F']:
                            field        = FIELD['FN']      #save attr
                            fid          = FIELD['FId']     #save attr
                            statement_1+='{2}name:"{0}", mag_id:"{1}"{3}, '.format(field,fid,LQ,RQ)
                    except:          #print('empty fields')
                        pass
                    statement_1+='],authors:['
                    for PERSON in ENTITY['AA']:  
                        author       = PERSON['AuN']    #save attr
                        author_id    = PERSON['AuId']
                        statement_1+='{2}name:"{0}", mag_id:"{1}"{3}, '.format(author,author_id,LQ,RQ)
                    statement_1+=']})._id;\n'
                    arango_cmd.write(statement_1)
                    for PERSON in ENTITY['AA']:
                        friend_num +=1  
                        author       = PERSON['AuN']    # if(new person) new person with MAG_id & upgrade person with MAG_id & build relation
                        author_id    = PERSON['AuId']
                        statement_2 ='    var mids = []\n' 
                        statement_2+='    if(person_id = db._query(`for per in person filter per.name_en=="{0}" return per._id`).toArray()[0]) {1}\n'.format(author,LQ)
                        statement_2+='        mids = mids.concat(db._document(person_id)["MAG_id"]) \n'
                        statement_2+='        mids.push("{0}")\n'.format(author_id)
                        statement_2+='        mids = Array.from(new Set(mids.filter( function(e){0} return e {1} )))\n'.format(LQ,RQ)
                        statement_2+='        db._update(person_id, {0}MAG_id: mids{1});  {1}\n'.format(LQ,RQ)
                        statement_2+='    else\n'
                        statement_2+='        person_id = db.person.save({2} name : "{0}", name_en:"{0}",MAG_id:"{1}" {3})._id\n'.format(author,author_id,LQ,RQ)
                        arango_cmd.write(statement_2)
                        statement_3 ='    db.author_of.save(person_id,paper_id,{})\n'  #save and print after every person_id(statement_2)
                        arango_cmd.write(statement_3)
                        try:
                            afn = PERSON['AfN']
                            afid= PERSON['AfId']
                            if afn=='shenyang':
                                afn = 'northeastern university'
                            affiliation_num +=1
                            statement_4 ='    if(org_id = db._query(`for school in org filter school.name_en=="{0}" return school._id`).toArray()[0]) \n '.format(afn)
                            statement_4+='        db._update(org_id,{1} MAG_id: "{0}" {2} ); \n'.format(afid,LQ,RQ)
                            statement_4+='    else\n'
                            statement_4+='        org_id = db.org.save({2} name:"{0}", name_en:"{0}", MAG_id:"{1}" {3})._id\n'.format(afn,afid,LQ,RQ)
                            statement_4+='    if(!(db._query(aql`for wo in work filter wo._from==${0}person_id{1} and wo._to==${0}org_id{1} return wo._id`).toArray()[0]))\n'.format(LQ,RQ) 
                            statement_4+='        db.work.save(person_id,org_id,{})\n'
                            arango_cmd.write(statement_4)
                        except:
                            emp_affiliation +=1   #print('empty affiliation')
                            pass

                    arango_cmd.write("}\n")
                    friend_num -=1
                #print(name_ch + ' paper : '+str(paper_cnt) + '  ,friend_num : ' + str(friend_num) +',  affiliation_num : ' + str(affiliation_num) + ' : ' + str(emp_affiliation)  )

            except KeyError:
                #print(exprs+':')
                #print(traceback.format_exc())
                pass
        arango_cmd.close()

    conn.close()
    print('executing .js file ....')
    f = os.popen(" arangosh --server.endpoint tcp://192.168.180.140:8529  --server.password 'pair' --javascript.execute paper_cmd; ")
    print('paper import OK!')


import 1..
import 2..
OK

PI & CO-PI : 82
arango OK!
王家慶 paper : 107  ,friend_num : 404,  affiliation_num : 477 : 34
曹昱 paper : 25  ,friend_num : 94,  affiliation_num : 102 : 17
鄧俊宏 paper : 48  ,friend_num : 90,  affiliation_num : 135 : 3
申雍 paper : 4  ,friend_num : 11,  affiliation_num : 10 : 5
謝易錚 paper : 5  ,friend_num : 15,  affiliation_num : 16 : 4
林芳邦 paper : 0  ,friend_num : 0,  affiliation_num : 0 : 0
徐國鎧 paper : 32  ,friend_num : 142,  affiliation_num : 165 : 9
楊靜瑩 paper : 12  ,friend_num : 34,  affiliation_num : 43 : 3
郭建宏 paper : 11  ,friend_num : 12,  affiliation_num : 22 : 1
李柏磊 paper : 35  ,friend_num : 189,  affiliation_num : 205 : 19
吳政瑋 paper : 0  ,friend_num : 0,  affiliation_num : 0 : 0
蘇柏齊 paper : 31  ,friend_num : 68,  affiliation_num : 95 : 4
張志文 paper : 16  ,friend_num : 26,  affiliation_num : 38 : 4
蔡宗翰 paper : 47  ,friend_num : 83,  affiliation_num : 125 : 5
呂菁菁 paper : 0  ,friend_num : 0,  affiliation_num : 0 : 0
鄭旭詠 paper : 33  ,friend_num : 59,  affiliation_

In [43]:
print('import 1..')
f = os.popen(" arangosh --server.endpoint tcp://192.168.180.140:8529  --server.password 'pair' --javascript.execute first_import_cmd; ")
time.sleep(3)

print('import 2..')
f = os.popen(" arangosh --server.endpoint tcp://192.168.180.140:8529  --server.password 'pair' --javascript.execute second_import_cmd; ")
time.sleep(3)

print('import 3..')
f = os.popen(" arangosh --server.endpoint tcp://192.168.180.140:8529  --server.password 'pair' --javascript.execute paper_cmd; ")
print('import ok')

import 1..
import 2..
import 3..
import ok


In [14]:
a = 'aaa/bbb/ccc'
print(a.split('/')[-1])

ccc
